In [3]:
import numpy as np # Linear algebra
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv)
np.random.seed(42)  
import matplotlib.pyplot as plt  # Matlab-style plotting
# Make sure plot shows immediately
%matplotlib inline 
import seaborn as sns # Library for plotting

color = sns.color_palette()
sns.set_style('darkgrid')

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
df = pd.read_csv('../data/processed/merged_data_Y2.csv')
df_y3 = pd.read_csv('../data/processed/merged_data_Y3.csv')

# 1 Evaluate the model on 3 different feature set.

In [5]:
# filter
feature_combination1 = df[['ClaimsCount', 'PayDelay', 'Unique_ProviderID', 'MED', 'EM', 'SCS', 'RAD', 'ANES', 'SDS', 'PL', 'SIS', 'SMS', 'SGS', 'SEOA', 'SNS',
 'SAS', 'SRS', 'SUS', 'SO', 'SMCD', 'Office', 'Independent Lab', 'Outpatient Hospital', 'Inpatient Hospital',
 'Urgent Care', 'Ambulance', 'Home', 'Other_y', 'Surgery', 'Internal', 'Laboratory', 'Pediatrics', 'Rehabilitation','Diagnostic Imaging', 'Anesthesiology', 'Emergency', 
        'General Practice', 'Other_x', 'Obstetrics and Gynecology', 'Pathology']]
# wrap
feature_combination2 = df[['ClaimsCount', 'Diagnostic Imaging', 'Emergency', 'Pathology',
       'Ambulance', 'Home', 'Inpatient Hospital', 'Office', 'Other_y',
       'Outpatient Hospital', 'Urgent Care', 'APPCHOL', 'CANCRM', 'CATAST',
       'LIVERDZ', 'PERINTL', 'PERVALV', 'PNCRDZ', 'RENAL1', 'SEPSIS', 'STROKE',
       '0', '1-2', '3-4', '5+', 'ANES', 'PL', 'SAS', 'SCS', 'SDS', 'SMCD',
       'SMS', 'SO']]
# embedded
feature_combination3 = df[['AgeAtFirstClaim', 'ClaimsCount', 'Unique_ProviderID', 'Unique_Vendor', 'PayDelay',
    'Diagnostic Imaging', 'Emergency', 'General Practice', 'Internal', 'Surgery',
    'Ambulance', 'Independent Lab', 'Inpatient Hospital', 'Office', 'Urgent Care',
    'MSC2a3', 'ROAMI', 'TRAUMA', '1-2', 'EM', 'MED', 'PL', 'RAD', 'SDS',
    '0- 1 month', 'DrugCount', 'LabCount']]

In [8]:
X_train1 = feature_combination1
y_train1 = df['DaysInHospital'] 

X_train2 = feature_combination2
y_train2 = df['DaysInHospital']  

X_train3 = feature_combination3
y_train3 = df['DaysInHospital']  

In [9]:
# filter
X_test1 = df_y3[['ClaimsCount', 'PayDelay', 'Unique_ProviderID', 'MED', 'EM', 'SCS', 'RAD', 'ANES', 'SDS', 'PL', 'SIS', 'SMS', 'SGS', 'SEOA', 'SNS',
 'SAS', 'SRS', 'SUS', 'SO', 'SMCD', 'Office', 'Independent Lab', 'Outpatient Hospital', 'Inpatient Hospital',
 'Urgent Care', 'Ambulance', 'Home', 'Other_y', 'Surgery', 'Internal', 'Laboratory', 'Pediatrics', 'Rehabilitation','Diagnostic Imaging', 'Anesthesiology', 'Emergency', 
        'General Practice', 'Other_x', 'Obstetrics and Gynecology', 'Pathology']]
y_test = df_y3['DaysInHospital'] 

# wrap
X_test2 = df_y3[['ClaimsCount', 'Diagnostic Imaging', 'Emergency', 'Pathology',
       'Ambulance', 'Home', 'Inpatient Hospital', 'Office', 'Other_y',
       'Outpatient Hospital', 'Urgent Care', 'APPCHOL', 'CANCRM', 'CATAST',
       'LIVERDZ', 'PERINTL', 'PERVALV', 'PNCRDZ', 'RENAL1', 'SEPSIS', 'STROKE',
       '0', '1-2', '3-4', '5+', 'ANES', 'PL', 'SAS', 'SCS', 'SDS', 'SMCD',
       'SMS', 'SO']]  
y_test = df_y3['DaysInHospital']

# embedded
X_test3 = df_y3[['AgeAtFirstClaim', 'ClaimsCount', 'Unique_ProviderID', 'Unique_Vendor', 'PayDelay',
    'Diagnostic Imaging', 'Emergency', 'General Practice', 'Internal', 'Surgery',
    'Ambulance', 'Independent Lab', 'Inpatient Hospital', 'Office', 'Urgent Care',
    'MSC2a3', 'ROAMI', 'TRAUMA', '1-2', 'EM', 'MED', 'PL', 'RAD', 'SDS',
    '0- 1 month', 'DrugCount', 'LabCount']] 
y_test = df_y3['DaysInHospital']

In [12]:
# filter
model = XGBRegressor(objective='reg:squarederror', random_state=42, max_depth=3)
model.fit(X_train1, y_train1)

y_pred = model.predict(X_test1)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Feature combination1:")
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

# wrap
model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=3)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Feature combination2:")
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

# embedded
model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=3)
model.fit(X_train3, y_train3)

y_pred = model.predict(X_test3)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Feature combination3:")
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

Feature combination1:
Mean Squared Error (MSE): 0.646256775799199
R-squared: 0.6770075664276588
Feature combination2:
Mean Squared Error (MSE): 0.6454602657525513
R-squared: 0.677405653887575
Feature combination3:
Mean Squared Error (MSE): 0.6756240973715153
R-squared: 0.66233008370353


### Conclusion
model on 27 feature sets perferm best, then 40 feature last 27 feature.

# 2 Hpyerparemeter tuning

## 2.1 max_depth 

In [11]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# k-fold 
def k_fold_cross_validation(X, y, k, model):
    """
    Implement k-fold cross validation.
    Returns the mean and standard deviation of the mean squared error (MSE).
    """
    np.random.seed(42)
    fold_size = len(X) // k
    indices = np.random.permutation(len(X))
    mse_scores = []

    for i in range(k):
        val_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate([indices[:i * fold_size], indices[(i + 1) * fold_size:]])

        X_train, X_val = X.iloc[train_indices], X.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mse_scores.append(mse)

    return np.mean(mse_scores), np.std(mse_scores)

# Implement nested cross validation and return the optimal hyperparameters
def nested_cross_validation(X, y, outer_k=3, inner_k=3, param_grid=None):
    """
    Implements nested cross validation for hyperparameter tuning (max_depth) and model evaluation.
    """
    if param_grid is None:
        param_grid = {'max_depth': [3, 5, 7, 9]}

    outer_mse_scores = []
    best_params = None
    best_mse = float('inf')

    np.random.seed(42)
    indices = np.random.permutation(len(X))
    fold_size_outer = len(X) // outer_k

    for i in range(outer_k):
        val_indices_outer = indices[i * fold_size_outer:(i + 1) * fold_size_outer]
        train_indices_outer = np.concatenate([indices[:i * fold_size_outer], indices[(i + 1) * fold_size_outer:]])

        X_train_outer, X_val_outer = X.iloc[train_indices_outer], X.iloc[val_indices_outer]
        y_train_outer, y_val_outer = y.iloc[train_indices_outer], y.iloc[val_indices_outer]

        for max_depth in param_grid['max_depth']:
            model = XGBRegressor(max_depth=max_depth, objective='reg:squarederror', random_state=42)

            # Perform k-fold cross validation on the inner layer
            mean_mse, _ = k_fold_cross_validation(X_train_outer, y_train_outer, inner_k, model)

            if mean_mse < best_mse:
                best_mse = mean_mse
                best_params = {'max_depth': max_depth}

        # Use the optimal hyperparameters to train the final model on the outer training set
        final_model = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
        final_model.fit(X_train_outer, y_train_outer)
        y_pred_outer = final_model.predict(X_val_outer)
        mse_outer = mean_squared_error(y_val_outer, y_pred_outer)
        outer_mse_scores.append(mse_outer)

    return np.mean(outer_mse_scores), np.std(outer_mse_scores), best_params

# run_multiple_experiments to return the optimal hyperparameters
def run_multiple_experiments(X, y, repetitions=20, outer_k=3, inner_k=3, param_grid=None):
    """
    Perform multiple nested cross-validation experiments to return the best hyperparameters and MSE statistics.
    """
    mse_results = []
    best_params_list = []

    for i in range(repetitions):
        print(f"Running repetition {i+1}/{repetitions}")
        mean_mse, std_mse, best_params = nested_cross_validation(X, y, outer_k=outer_k, inner_k=inner_k, param_grid=param_grid)
        mse_results.append(mean_mse)
        best_params_list.append(best_params)

    return np.mean(mse_results), np.std(mse_results), best_params_list

param_grid = {'max_depth': [3, 5, 7, 9]}
mean_mse, std_mse, best_params_list = run_multiple_experiments(X_train2, y_train2, repetitions=20, param_grid=param_grid)

# result
print(f"Mean MSE after 20 repetitions: {mean_mse}")
print(f"Standard deviation of MSE after 20 repetitions: {std_mse}")
print(f"Best max_depth for each run: {best_params_list}")

Running repetition 1/20
Running repetition 2/20
Running repetition 3/20
Running repetition 4/20
Running repetition 5/20
Running repetition 6/20
Running repetition 7/20
Running repetition 8/20
Running repetition 9/20
Running repetition 10/20
Running repetition 11/20
Running repetition 12/20
Running repetition 13/20
Running repetition 14/20
Running repetition 15/20
Running repetition 16/20
Running repetition 17/20
Running repetition 18/20
Running repetition 19/20
Running repetition 20/20
Mean MSE after 20 repetitions: 0.7446449710175636
Standard deviation of MSE after 20 repetitions: 2.220446049250313e-16
Best max_depth for each run: [{'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}, {'max_depth': 3}]

### Try different max_depth on y_test

In [15]:
model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=2)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=3)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=4)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=5)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=6)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=7)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')


model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=8)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=9)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,max_depth=10)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

Mean Squared Error (MSE): 0.6570902801744934
R-squared: 0.6715930933059406
Mean Squared Error (MSE): 0.6454602657525513
R-squared: 0.677405653887575
Mean Squared Error (MSE): 0.6497065141915417
R-squared: 0.6752834229598356
Mean Squared Error (MSE): 0.6571138015014533
R-squared: 0.6715813375906893
Mean Squared Error (MSE): 0.6803883038051081
R-squared: 0.659948982742346
Mean Squared Error (MSE): 0.7243535166095867
R-squared: 0.6379756253896385
Mean Squared Error (MSE): 0.7564328144576934
R-squared: 0.6219427250514742
Mean Squared Error (MSE): 0.796518014707727
R-squared: 0.6019085577300071
Mean Squared Error (MSE): 0.79393285082255
R-squared: 0.6032005958265114


### Conclusion
Best max_depth is 3

## 2.2 subsample

In [13]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
# same as max_depth
def k_fold_cross_validation(X, y, k, model):
    """
    Implement k-fold cross validation.
    Returns the mean and standard deviation of the mean squared error (MSE).
    """
    np.random.seed(42)
    fold_size = len(X) // k
    indices = np.random.permutation(len(X))
    mse_scores = []

    for i in range(k):
        val_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate([indices[:i * fold_size], indices[(i + 1) * fold_size:]])

        X_train, X_val = X.iloc[train_indices], X.iloc[val_indices]
        y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mse_scores.append(mse)

    return np.mean(mse_scores), np.std(mse_scores)

def nested_cross_validation(X, y, outer_k=3, inner_k=3, param_grid=None):
    """
    Implements nested cross validation for hyperparameter tuning (subsample) and model evaluation.
    """
    if param_grid is None:
        param_grid = {'subsample': [0.3, 0.5, 0.7, 1.0]}

    outer_mse_scores = []
    best_params = None
    best_mse = float('inf')

    np.random.seed(42)
    indices = np.random.permutation(len(X))
    fold_size_outer = len(X) // outer_k

    for i in range(outer_k):
        val_indices_outer = indices[i * fold_size_outer:(i + 1) * fold_size_outer]
        train_indices_outer = np.concatenate([indices[:i * fold_size_outer], indices[(i + 1) * fold_size_outer:]])

        X_train_outer, X_val_outer = X.iloc[train_indices_outer], X.iloc[val_indices_outer]
        y_train_outer, y_val_outer = y.iloc[train_indices_outer], y.iloc[val_indices_outer]

        for subsample in param_grid['subsample']:
            model = XGBRegressor(subsample=subsample, objective='reg:squarederror', random_state=42)

            mean_mse, _ = k_fold_cross_validation(X_train_outer, y_train_outer, inner_k, model)

            if mean_mse < best_mse:
                best_mse = mean_mse
                best_params = {'subsample': subsample}

        final_model = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
        final_model.fit(X_train_outer, y_train_outer)
        y_pred_outer = final_model.predict(X_val_outer)
        mse_outer = mean_squared_error(y_val_outer, y_pred_outer)
        outer_mse_scores.append(mse_outer)

    return np.mean(outer_mse_scores), np.std(outer_mse_scores), best_params

def run_multiple_experiments(X, y, repetitions=20, outer_k=3, inner_k=3, param_grid=None):
    """
    Perform multiple nested cross-validation experiments to return the best hyperparameters and MSE statistics.
    """
    mse_results = []
    best_params_list = []

    for i in range(repetitions):
        print(f"Running repetition {i+1}/{repetitions}")
        mean_mse, std_mse, best_params = nested_cross_validation(X, y, outer_k=outer_k, inner_k=inner_k, param_grid=param_grid)
        mse_results.append(mean_mse)
        best_params_list.append(best_params)

    return np.mean(mse_results), np.std(mse_results), best_params_list

param_grid = {'subsample': [0.3, 0.5, 0.7, 1.0]}
mean_mse, std_mse, best_params_list = run_multiple_experiments(X_train2, y_train2, repetitions=20, param_grid=param_grid)

print(f"Mean MSE after 20 repetitions: {mean_mse}")
print(f"Standard deviation of MSE after 20 repetitions: {std_mse}")
print(f"Best subsample for each run: {best_params_list}")


Running repetition 1/20
Running repetition 2/20
Running repetition 3/20
Running repetition 4/20
Running repetition 5/20
Running repetition 6/20
Running repetition 7/20
Running repetition 8/20
Running repetition 9/20
Running repetition 10/20
Running repetition 11/20
Running repetition 12/20
Running repetition 13/20
Running repetition 14/20
Running repetition 15/20
Running repetition 16/20
Running repetition 17/20
Running repetition 18/20
Running repetition 19/20
Running repetition 20/20
Mean MSE after 20 repetitions: 0.7898648347938104
Standard deviation of MSE after 20 repetitions: 1.1102230246251565e-16
Best subsample for each run: [{'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}, {'subsample': 1.0}

### Try different subsample on y_test

In [14]:
model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.1,max_depth=3)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.3,max_depth=3)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.5,max_depth=3)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.7,max_depth=3)
model.fit(X_train2, y_train2)
y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=0.9,max_depth=3)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

model = XGBRegressor(objective='reg:squarederror', random_state=42,subsample=1,max_depth=3)
model.fit(X_train2, y_train2)

y_pred = model.predict(X_test2)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared: {r2}')

Mean Squared Error (MSE): 0.8093876450630393
R-squared: 0.5954764499621648
Mean Squared Error (MSE): 0.6964396319051389
R-squared: 0.6519266954422467
Mean Squared Error (MSE): 0.6674704245153356
R-squared: 0.6664052048271938
Mean Squared Error (MSE): 0.6680623170241516
R-squared: 0.6661093830903938
Mean Squared Error (MSE): 0.6518017759993782
R-squared: 0.6742362328403241
Mean Squared Error (MSE): 0.6454602657525513
R-squared: 0.677405653887575


### Conclusion
Best subsample is 1